In [1]:
from nltk import word_tokenize
import nltk
import threading
import time
from operator import itemgetter
from ecommercetools import seo

In [2]:
def process(content):
    #------- NORMALIZATION ----------------#
    normalize = []
    for line in content:
        normalize.append(' '.join([''.join([ch.lower() for ch in word if ch.isalpha()]) for word in line.split()]))
    #------- TOKENIZATION ----------------#
    tokenize = []
    for line in normalize:
        tokenize.append(nltk.word_tokenize(line))
    #------- STOPWORD REMOVAL ----------------#
    stopword_lst = nltk.corpus.stopwords.words(fileids='english')
    nostop = []
    for line in tokenize:
        nostop.append([word for word in line if word not in stopword_lst])
    #------- Bi-Gram------------------------#
    newData = []
    for i in range(len(nostop)):
        data = []
        for bi in list(nltk.bigrams(nostop[i])):
            ls = list(bi)
            ls.sort()
            data.append(' '.join(ls))
        newData.append(data)
    #------- Word Frequency ----------------#
    wordFreq = {}
    for line in newData:
        for word in line:
            if(word not in wordFreq.keys()):
                wordFreq[word] = 0
            wordFreq[word] += 1
    return wordFreq

In [3]:
def findTopic(question, k = 1):
    try:
        results = seo.get_serps(question)
    except:
        return ("",[])
    wordDict = process(results.title)
    res = dict(sorted(wordDict.items(), key = itemgetter(1), reverse = True)[:k])
    res = list(res.keys())
    res.sort()
    res = ' '.join(res)
    return (res,list(results.link)[:3])

In [4]:
with open('paper2.txt') as f:
    data = f.read()
import ast
paper = ast.literal_eval(data)
paper

{'P1.Q1': 'Q.1 Explain the following file allocation methods with suitable diagram(s): (5) (a) Linked allocation (b) Indexed allocation',
 'P1.Q2': 'Q.2 Differentiate between peer-to-peer and client-server computing (5) environments.',
 'P1.Q3': "Q.3 (a) Write and explain the pseudo-code/algorithm/program of (5) Peterson's Solution used for accessing the critical section by two co-operating processes.  (5)",
 'P1.Q4': 'Q.4 On a system with n CPUs, what is the minimum number of processes that (5) can be in the ready, run, and blocked states? Explain with a suitable process state diagram.',
 'P1.Q5': 'Q.5 A system has three processes and four identical resources. Each process (5) needs a maximum of three resources to complete. Is deadlock possible? Explain your answer.',
 'P2.Q1': 'Q.6 Define concurrency and race condition.',
 'P2.Q2': 'Q.7 Consider a computer system with 264bits virtual address space and page (5) size of 1 Kbyte. The physical memory is of 64 Kbytes and the size of page 

In [5]:
def fetchOnlineAns(paper):
    paper_topics = {}
    topic_qnNo = {}
    def searchTopic(key):
        paper_topics[key] = findTopic(paper[key])
        if len(paper_topics[key][1])==0:
            return
        if paper_topics[key][0] not in topic_qnNo.keys():
            topic_qnNo[paper_topics[key][0]] = []
        topic_qnNo[paper_topics[key][0]].append(key)

    threads=[]
    for key in list(paper.keys()):
        t = threading.Thread(target=searchTopic, args=[key])
        t.start()
        threads.append(t)
    for t in threads:
        t.join()
        time.sleep(0.1)
    marklist = sorted(topic_qnNo.items(), key=lambda x:len(x[1]), reverse=True)
    topicData = [[tup[0],' '.join(tup[1])] for tup in marklist]
    qNumbers=list(paper.keys())
    qNumbers.sort()
    linkData = [[qn,paper[qn]]+paper_topics[qn][1] for qn in qNumbers]
    return {'links':linkData,'topics':topicData}

In [6]:
data={
  "P1.Q1": "(b) \"Transactions; the basic units of human communication, are outcomes of our ego-states Explain this statement in the light of",
  "P1.Q3": "What is the main objective of writing a Proposal? What structure sho be followed for writing an effective proposal? (300-350 wor",
  "P1.Q2": "2 Good business writing involves selecting the right words and for correct sentences: Illustrate with the help of suitable examples:",
  "P1.Q5": "A friend of yours is the Managing Director of Shah Textile Corporation Shaheed Marg; Surat He has suggested that you apply for the post of ar Engineer in the sister concern called Mehta Engineering Works, Gokhale Marg, Pune",
  "P1.Q8": "Q6 A popular advertisement says: \"You no longer need words to 10 communicate. Justify   the statement in the context   of   non-verbal communication: (300-350 words)",
  "P1.Q6": "Dai &co TUi Uceo ALLCLLL=40 C~C ACECL (Assume any other necessary details ) (300-350 wor",
  "P1.Q7": "(a) When you are sending a message, what are the possible languag related barriers that you might face? (125- 150 words (b) How can an organization ensure that both upward and downwar channels of communication are effective? (125-150 words",
  "P1.Q4": "be followed for writing an effective proposal? (300-350 words) 04 A friend of yours is the Managing Director of Shah Textile Corporation; Shaheed Marg Surat He has suggested that you apply for the post of an Engineer in the sister concern called Mehta Engineering Works, Gokhale Marg Pune Draft a resume for the job Attach it with a Cover letter: (Assume any other necessary details ) (300-350 words) Q5 (a) When you are sending a message; what are the possible language- related barriers that you might face? (125-150 words)"
}

In [21]:
data={
  "P1.Q3": "2 Define a report? What are the characteristics ofa good report? (150 words)",
  "P1.Q2": "(b) What should you keep in mind while drafting a sentence? (150 words",
  "P1.Q1": "21 (a) What should you keep in mind while choosing yOur words for business writing? (150 Words) (5)",
  "P1.Q4": "Q3 What are the different types of proposals? Explain the 5Ws and 1H as princip proposal writing? (150 words)",
  "P1.Q6": "25 Why is training in Non-Verbal Communication necessary for professions like engineering? How can it be used effectively to one'$ advantage? (300 wor",
  "P1.Q7": "Q6 Distinguish between formal and informal communication networks in organizations? Whatare the objectives of downward and upward channels communication? (300 words)",
  "P1.Q5": "Attach it with & cover letter: (250 words) Wanted a Deputy General Manager (Operations) at our new office in Place, New Delhi, India: Engineering graduates with minimum 7 years experience can apply: Salary is negotiable: Apply with particulars to 650 Puni Hlovd td New Delhi 110001 latest by 7th December, 2016."
}

In [22]:
%%time
fetchOnlineAns(data)

CPU times: user 590 ms, sys: 44.3 ms, total: 634 ms
Wall time: 4.27 s


{'links': [['P1.Q1',
   '21 (a) What should you keep in mind while choosing yOur words for business writing? (150 Words) (5)',
   'https://www.wordstream.com/blog/ws/2014/08/07/improve-writing-skills',
   'https://self-publishingschool.com/how-to-write-a-book/',
   'https://ieltsliz.com/how-many-words-ielts-writing/'],
  ['P1.Q2',
   '(b) What should you keep in mind while drafting a sentence? (150 words',
   'https://opentextbc.ca/writingforsuccess/chapter/chapter-3-putting-ideas-into-your-own-words-and-paragraphs/',
   'https://saylordotorg.github.io/text_business-english-for-success/s11-the-writing-process-how-do-i-b.html',
   'https://www.scribbr.com/academic-writing/writing-process/'],
  ['P1.Q3',
   '2 Define a report? What are the characteristics ofa good report? (150 words)',
   'https://www.lkouniv.ac.in/site/writereaddata/siteContent/202004080628301891arvind_BRM_Report_Writing.pdf',
   'https://www.yourarticlelibrary.com/company/drafting/top-11-characteristics-of-a-good-repor